In [70]:
#Import
from spotify import Client, OAuth
import pandas as pd

In [71]:
auth = OAuth('f716b10140c04e9997c89e1d094745f6', 'b9aa1c134fe244ebaed7b3cba541f6a7')
auth.request_client_credentials()

In [72]:
spotifyClient = Client(auth)

In [73]:
spotifyAPI = spotifyClient.api

In [74]:
def getTrackAttributes(trackIdList = []):
    trackJson = spotifyAPI.tracks_audio_features(trackIdList)
    trackList = trackJson['audio_features']
    trackList = list(filter(None, trackList))
    return trackList

In [75]:
def getAttributeDF(trackFeatList):
    trackDF = pd.DataFrame(trackFeatList)
    trackDF = trackDF.drop(['analysis_url', 'track_href', 'type', 'uri'], axis=1)
    return trackDF

In [76]:
def getPlaylistTrack(play_id,user_id):
    tracks = spotifyAPI.user_playlist_tracks(playlist_id= play_id, user_id= user_id,fields='items(track.id,track.album.name,track.name,track.artists)')
    tracks = list(filter(None, tracks))
    return tracks


In [77]:
def trackJSONtoDF(tracks,genre):
    flatData = []

    for i in range(len(tracks['items'])):
        trackObj = {}
        trackObj['id'] = tracks['items'][i]['track']['id']
        trackObj['album'] = tracks['items'][i]['track']['album']['name']
        trackObj['track_name'] = tracks['items'][i]['track']['name']
        trackObj['artist'] = tracks['items'][i]['track']['artists'][0]['name']
        trackObj['genre'] = genre
        flatData.append(trackObj)
    return pd.DataFrame(flatData)

In [80]:
def pipeLineFunc(play_id,user_id,genre):
    # Fetch playlist tracks
    plTracks = getPlaylistTrack(play_id,user_id)
    
    
    # Convert to DF
    plTracksDF =  trackJSONtoDF(plTracks,genre)

    # Get list of track ids
    trackIds = list(plTracksDF['id'])

    # Get tracks features
    trackFeat = getTrackAttributes(trackIds)

    # Convert JSON to DF
    trackFeatDF = getAttributeDF(trackFeat)

    # Merging 2 data frames
    resultDF = pd.concat([plTracksDF, trackFeatDF], axis=1, join='inner')

    # Droping duplicate ID columns
    resultDF = resultDF.T.drop_duplicates().T

    return resultDF
    
    

In [82]:
res = pipeLineFunc('2kpoUUJ5a4Cw3feTkFJhZ2','1276640268','top100')
res

TypeError: list indices must be integers or slices, not str

In [12]:
res.to_csv('billboardTop100.csv'

In [13]:
res.columns

Index(['album', 'artist', 'genre', 'id', 'track_name', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence'],
      dtype='object')

In [65]:
playListData = [{'user':'spotify','playlist_id':'37i9dQZF1DWTwnEm1IYyoj','genre':'Pop'},
{'user':'1154148597','playlist_id':'03Wky4J3HehgLEqgkYmia6','genre':'Pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5CHqBODmn2c','genre':'Pop'},
{'user':'1123398699','playlist_id':'23o2085ZcSb5V3yVxnHj9c','genre':'Pop'},
{'user':'1180623169','playlist_id':'2yTNGJu0CWKIms9vhpOPeY','genre':'Pop'},
{'user':'1288831866','playlist_id':'0lvwSeY2Og8XyileVyWZ1v','genre':'Pop'},
{'user':'dominic.rupp','playlist_id':'2zu4FqqzubZK4oXAZFEL4L','genre':'Pop'},
{'user':'redmusiccompany','playlist_id':'3U25AXqlklo3o6LqM1wX1I','genre':'Country'},
{'user':'1276364988','playlist_id':'7fIdjWhWzPbf1g5OSDKOXw','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DX13ZzXoot6Jc','genre':'Country'},
{'user':'spotify','playlist_id':'37i9dQZF1DX68H8ZujdnN7','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DWZn9s1LNKPiM','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DWWwzidNQX6jx','genre':'Rock'},
{'user':'sheffield806','playlist_id':'45vvbQl0GL7Niubb3H0U2Y','genre':'Rock'},
{'user':'redmusiccompany','playlist_id':'3yhmzZlOnRMCFV7PuMFPKy','genre':'Rock'},
{'user':'spotify','playlist_id':'37i9dQZF1DX12YIlzNkccL','genre':'Classical'},
{'user':'spotify','playlist_id':'37i9dQZF1DX8qqIDAkKiQg','genre':'Classical'},
{'user':'1269890947','playlist_id':'6Jn7zgkmA29infIRmptveP','genre':'Classical'},
{'user':'merovingin','playlist_id':'0ZRwrJ2EDGyKR6YgQPWXeO','genre':'Hip Hop'},
{'user':'akime1212','playlist_id':'2eY9lPvBKkluNSpeYJo71a','genre':'Hip Hop'},
{'user':'19jconnell79','playlist_id':'5772HGqmp2E99GQo5tfmcJ','genre':'Hip Hop'},
{'user':'1113721529','playlist_id':'0t3Mn6bXtPNWWZK6x9DtBH','genre':'Hip Hop'},
{'user':'playstation_music','playlist_id':'2NoR0KhNZ8oZnC3HoNt2FV','genre':'Hip Hop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX8GjsySWIS1x','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DWYnY33e7En9e','genre':'Gaming'},
{'user':'1274440778','playlist_id':'5fdipI1K5g6ij4gCP8SvLj','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DX6AIf8KmeDe7','genre':'Gaming'},
{'user':'emma.ehrhardt','playlist_id':'0QApAL6CpODIMyweRdAvM4','genre':'Gaming'},
{'user':'spotify','playlist_id':'37i9dQZF1DX0SM0LYsmbMT','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWVqfgj8NZEp1','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DX7YCknf2jT6s','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWVzZlRWgqAGH','genre':'Jazz'},
{'user':'starbucks','playlist_id':'2UPjWBRjJqMQY3LRV2aDTS','genre':'Jazz'},
{'user':'spotify','playlist_id':'37i9dQZF1DWSiyIBdVQrkk','genre':'reggae'},
{'user':'boogalooboo','playlist_id':'4gRio043A2No9SRU7ERq6q','genre':'reggae'},
{'user':'constantinex','playlist_id':'0s8tfHVdQa8duFMev3ZlgD','genre':'reggae'},
{'user':'constantinex','playlist_id':'53iDeoyHodextygU8wpras','genre':'reggae'},
{'user':'jmelendez1024','playlist_id':'0ka9e6tQIfLXVSNXraZxNz','genre':'reggae'},
{'user':'spotify','playlist_id':'37i9dQZF1DX2ohL85TE8TI','genre':'K-pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DXbirtHQBuwCo','genre':'K-pop'},
{'user':'spotify','playlist_id':'37i9dQZF1DX5eq3AONkaho','genre':'K-pop'},
{'user':'thehela1997','playlist_id':'4c1DULvTgCGWcTETJrra5A','genre':'K-pop'},
{'user':'12165116393','playlist_id':'3OWkXYSwYzKeN64mCqkouR','genre':'K-pop'}]

In [66]:
playlistDF = pd.DataFrame(playListData)

In [67]:
playlistDF["playlist_id"][1]

'03Wky4J3HehgLEqgkYmia6'

In [68]:
trackDFMain = pd.DataFrame()
for i in range(len(playlistDF)):
    resDF = pipeLineFunc(playlistDF["playlist_id"][i],playlistDF["user"][i],playlistDF["genre"][i])
    trackDFMain = trackDFMain.append(resDF)

In [38]:
trackDFMain = pd.DataFrame()

In [63]:
res = pipeLineFunc('37i9dQZF1DWTwnEm1IYyoj','spotify','Pop')
trackDFMain = trackDFMain.append(res)


In [64]:
plTracks = getPlaylistTrack('37i9dQZF1DWTwnEm1IYyoj','spotify')
    
plTracks

['items']

In [69]:
    # Convert to DF
trackDFMain

""
